### Importing dependencies

In [1]:
import tensorflow as tf
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()
from tensorflow import keras
# from tensorflow import keras.preprocessing.imageImageDataGenerator 
# import tf.contrib.keras.preprocessing.image.ImageDataGenerator as ImageDataGenerator
# # keras.preprocessing.image.ImageDataGenerator
# import keras
# from keras import layers
# from keras.preprocessing import image # keras.preprocessing.image
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import ResNet50
# from keras.applications.resnet50 import preprocess_input
# from keras.callbacks import ModelCheckpoint
# from keras.models import load_model #  keras.models.load_model

import matplotlib.pyplot as plt
import os, shutil
from livelossplot import PlotLossesKeras
import numpy as np

tf.__version__

Using TensorFlow backend.


'1.8.0'

In [2]:
# from tensorflow.python.client import device_lib

# device_lib.list_local_devices()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '2' 

In [4]:
traindir ='/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train'
valdir = '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation'

batch_size = 32 #32
img_size = 224
epochs = 3

### Create Keras data generators 

In [5]:
#todo: figure out how to add preprocessing.

In [6]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=keras.applications.resnet50.preprocess_input,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = keras.preprocessing.image.ImageDataGenerator()
#     preprocessing_function=keras.applications.resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    traindir,
    batch_size=batch_size,
    class_mode='binary',
    target_size=(img_size,img_size))

validation_generator = validation_datagen.flow_from_directory(
    valdir,
    shuffle=False,
    class_mode='binary',
    target_size=(img_size,img_size))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Import pre-trained ResNet50

Time to load the pre-trained ResNet model. We'll freeze all layers, so there's no backpropagation of gradients through the ResNet50 layers (this saves us A LOT of time and resources)

In [7]:
keras.backend.set_learning_phase(0) # see https://github.com/keras-team/keras/issues/9214  and https://github.com/keras-team/keras/pull/9965

conv_base = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet')

# freeze layers in resnet50
for layer in conv_base.layers:
    layer.trainable = False

### Stack trainable, fully-connected (dense) layers on top of it

These are the layers, which will have randomly initiated weights: these are the ones we'll actually train.

In [10]:
model = conv_base.output
keras.backend.set_learning_phase(1)
model = keras.layers.GlobalAveragePooling2D()(model)

# add fc layers
model = keras.layers.Dense(128, activation='relu')(model) 
predictions = keras.layers.Dense(1, activation='sigmoid')(model)

In [36]:
len(conv_base.variables)

318

### Prepare model checkpointing

In [11]:
model = keras.Model(conv_base.input, predictions)

In [12]:
model_checkpoint_dir = "./output"

if not os.path.exists(model_checkpoint_dir):
    os.mkdir(model_checkpoint_dir)
    print('Created directory: ', model_checkpoint_dir)

model_checkpoint_dir = model_checkpoint_dir + "/best_model.hdf5"


Keras API allows us to easily save the model architecture at the stage it reaches the best validation accuracy (or any metric we want to optimize for). The saving is done through ModelCheckpoint callback.

In [13]:
checkpoint_save_best = keras.callbacks.ModelCheckpoint(model_checkpoint_dir, monitor='val_acc',
verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

In [14]:
len(model.trainable_weights)

4

### Compile and train the model (A)

If eager mode wasn't enabled, we could compile train the model using the standard Keras API, like so:

In [16]:
# optimizer = keras.optimizers.RMSprop()

# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

ValueError: Only TF native optimizers are supported in Eager mode.

But since we want to see how eager execution works, we'll do it differently.

Note that compiling a tf.keras model while tfe.enable_eager_execution() results in: 
"ValueError: Only TF native optimizers are supported in Eager mode."


In [14]:
# %%time

# history = model.fit_generator(generator=train_generator,
#                     epochs=epochs,
#                     validation_data=validation_generator, 
#                     steps_per_epoch = train_generator.n // batch_size,
#                     validation_steps = validation_generator.n // batch_size,
#                     initial_epoch=0,
#                     callbacks=[checkpoint_save_best,])
#                     #          plot_losses])    

### Train the model (B)

In [15]:
# tensorflow eager execution

In [17]:
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# optimizer = tf.keras.optimizers.RMSprop()
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)


In [18]:
def loss(model, inputs, labels):
    prediction = model(inputs)
    
    return tf.losses.sigmoid_cross_entropy(logits=prediction, multi_class_labels=tf.expand_dims(labels, dim=1))

In [19]:
def grad(model, inputs, targets):

    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

Let's print the loss on a single batch to see if it works so far

In [20]:
#test forward prop
with tf.device("/gpu:0"):
    x, y = iter(train_generator).next()
    print("Initial loss: {:.3f}".format(loss(model, x, y)))

Initial loss: 0.623


In [21]:
#there's something wrong with the below- it's suspiciously slow, compared to the pure keras version...

In [51]:
len(model.trainable_variables)

4

In [53]:
model.trainable_variables

[<tf.Variable 'dense_2/kernel:0' shape=(2048, 128) dtype=float32, numpy=
 array([[-0.01560671,  0.03894274,  0.01391688, ..., -0.00737522,
         -0.00492744,  0.0169412 ],
        [-0.01842074,  0.05016192,  0.04371505, ...,  0.01060552,
         -0.02638582, -0.01518007],
        [ 0.03870697, -0.00990153,  0.0372727 , ...,  0.0393369 ,
          0.0081052 , -0.03920021],
        ...,
        [-0.00633602,  0.03693205, -0.04665111, ..., -0.03187395,
         -0.05339753, -0.00830605],
        [-0.01559854, -0.02060005,  0.01769498, ..., -0.04769189,
         -0.01922006,  0.03929759],
        [-0.01189886, -0.02948019, -0.00610928, ...,  0.02218145,
         -0.04089361,  0.04934127]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(128,) dtype=float32, numpy=
 array([ 0.05357934, -0.01406946, -0.02029084,  0.07233132, -0.0009378 ,
        -0.00211341,  0.06436323, -0.00967983,  0.07279093,  0.06702536,
        -0.00259931, -0.00514135,  0.00640002, -0.01605608,  0.06778348,

In [52]:
with tf.device("/gpu:0"):

    # for epoch in range(epochs):
    for (batch, (inputs, labels)) in enumerate(train_generator):

        grads = grad(model, inputs, labels)

        optimizer.apply_gradients(zip(grads, model.variables), #changingto model.trainable_variables doesn't improve speed
                                    global_step=tf.train.get_or_create_global_step())
#         if batch % 200 == 0:
#         print("Loss at step {:04d}: {:.3f}".format(batch, loss(model, x, y)))
        print("Loss at step {:04d}: {:.3f}".format(batch, loss(model, x, y)))
        
print("Final loss: {:.3f}".format(loss(model, x, y)))

Loss at step 0000: 0.625
Loss at step 0001: 0.624
Loss at step 0002: 0.624
Loss at step 0003: 0.624
Loss at step 0004: 0.623
Loss at step 0005: 0.623
Loss at step 0006: 0.622
Loss at step 0007: 0.623
Loss at step 0008: 0.623
Loss at step 0009: 0.622
Loss at step 0010: 0.622
Loss at step 0011: 0.622
Loss at step 0012: 0.622
Loss at step 0013: 0.623
Loss at step 0014: 0.622
Loss at step 0015: 0.621
Loss at step 0016: 0.621
Loss at step 0017: 0.621
Loss at step 0018: 0.621
Loss at step 0019: 0.621
Loss at step 0020: 0.621
Loss at step 0021: 0.620
Loss at step 0022: 0.620
Loss at step 0023: 0.619
Loss at step 0024: 0.619
Loss at step 0025: 0.619
Loss at step 0026: 0.619
Loss at step 0027: 0.619
Loss at step 0028: 0.618
Loss at step 0029: 0.618
Loss at step 0030: 0.619
Loss at step 0031: 0.619
Loss at step 0032: 0.617
Loss at step 0033: 0.613
Loss at step 0034: 0.614
Loss at step 0035: 0.604
Loss at step 0036: 0.606
Loss at step 0037: 0.589
Loss at step 0038: 0.603
Loss at step 0039: 0.606


KeyboardInterrupt: 

In [ ]:
stop

### Load the best model and evaluate it on a sample image 

In [ ]:
keras.backend.clear_session()

In [ ]:
loaded_model = keras.models.load_model(model_checkpoint_dir, compile=False) #compile causes issues if not turned off - we are not training the model so we don't need to compile the model #the workaround to this user warning error might be to set all weights to not-trainable explicitly (in a loop) https://stackoverflow.com/questions/49195189/error-loading-the-saved-optimizer-keras-python-raspberry
print("Model loaded successfully!")

In [ ]:
test_cat = "/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/test/cats/cat.1505.jpg"

In [ ]:
img = image.load_img(test_cat, target_size=(img_size, img_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = loaded_model.predict(x)
preds
# pred_class = loaded_model.predict_classes(x)
# pred_class = loaded_model.probas_to_classes(x)

if preds[0][0] < 0.5:
    print("IT'S A CAT! ({:.2})".format(preds[0][0]))
else:
    print("IT'S A DOG! ({:.2})".format(preds[0][0]))

In [ ]:
img = img.resize((128,128))
plt.imshow(img)
plt.show()